In [145]:
import pandas as pd
import numpy as np
import math

In [107]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [108]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [109]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Попробуем предсказывать стоимость билета (Fare) и построим регрессию.

В качестве обучающей выборки возьмем номер класса (Pclass), пол (Sex), класс каюты (Cabin) и порт посадки (Embarked)

In [110]:
X_train = df_train[["Pclass", "Sex", "Cabin", "Embarked"]]
X_test = df_test[["Pclass", "Sex", "Cabin", "Embarked"]]

X_train = X_train.fillna("N")
X_test = X_test.fillna("N")

Y_train = df_train.Fare
Y_test = df_test.Fare


In [111]:
X_train.head()

,Pclass,Sex,Cabin,Embarked
0,3,male,N,S
1,1,female,C85,C
2,3,female,N,S
3,1,female,C123,S
4,3,male,N,S


In [112]:
Y_train.head()

0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64

In [113]:
def replace_cabin(cabin):
    return cabin[0]

In [114]:
X_train.Cabin = X_train.Cabin.apply(replace_cabin)
X_test.Cabin = X_test.Cabin.apply(replace_cabin)

In [115]:
X_train.head()

,Pclass,Sex,Cabin,Embarked
0,3,male,N,S
1,1,female,C,C
2,3,female,N,S
3,1,female,C,S
4,3,male,N,S


In [116]:
def create_dict(column):
    return dict([(c, i) for i, c in enumerate(column.unique())])

In [117]:
sex_label2id = create_dict(X_train.Sex)
cabin_label2id = create_dict(X_train.Cabin)
embarked_label2id = create_dict(X_train.Embarked)
p_class_label2id = create_dict(X_train.Pclass)

In [118]:
def vectorize_categorical(value, dictionary):
    return np.eye(len(dictionary))[dictionary[value]]

In [119]:
X_train.Sex = X_train.Sex.apply(vectorize_categorical, dictionary=sex_label2id)
X_train.Cabin = X_train.Cabin.apply(vectorize_categorical, dictionary=cabin_label2id)
X_train.Embarked = X_train.Embarked.apply(vectorize_categorical, dictionary=embarked_label2id)
X_train.Pclass = X_train.Pclass.apply(vectorize_categorical, dictionary=p_class_label2id)

In [120]:
X_train.head()

,Pclass,Sex,Cabin,Embarked
0,"[1.0, 0.0, 0.0]","[1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
1,"[0.0, 1.0, 0.0]","[0.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]"
2,"[1.0, 0.0, 0.0]","[0.0, 1.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
3,"[0.0, 1.0, 0.0]","[0.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
4,"[1.0, 0.0, 0.0]","[1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"


In [325]:
X_test.Sex = X_test.Sex.apply(vectorize_categorical, dictionary=sex_label2id)
X_test.Cabin = X_test.Cabin.apply(vectorize_categorical, dictionary=cabin_label2id)
X_test.Embarked = X_test.Embarked.apply(vectorize_categorical, dictionary=embarked_label2id)
X_test.Pclass = X_test.Pclass.apply(vectorize_categorical, dictionary=p_class_label2id)

### Попробуем имплементировать  линейную регрессию самостоятельно

$f(x) = w_0x_0 + w_1x_1 + w_2x_2 + w_3x_3 + w_4x_4 + w_5x_5 + b$

правда, кажется, признаков-то не 5

Функция для инициализации весов:

In [86]:
def init(n):
    return {"w": np.zeros(n), "b": 0.0}

Предсказываем стоимость билета:

In [264]:
def predict(x, parameters):
    return np.matmul(np.array(x), parameters['w']) + parameters["b"]

In [266]:
# Initialize model parameters
n = len(np.concatenate(X_train.iloc[0]))
print(f"Всего признаков:{n}")
model_parameters = init(n)

# Make prediction for every data sample
X = [np.concatenate(line) for line in X_train.values]
predictions = predict(X, model_parameters)
print(f"Предсказания: {predictions}, всего {len(predictions)}")

Всего признаков:18
Предсказания: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.

Посчитаем ошибку:

In [184]:
def mse(predictions, targets):
    # Retrieving number of samples in dataset
    samples_num = len(predictions)
    
    # Summing square differences between predicted and expected values
    accumulated_error = 0.0
    for prediction, target in zip(predictions, targets):
        accumulated_error += (prediction - target)**2
        
    # Calculating mean
    mse_error = (1.0 / samples_num) * accumulated_error
    return mse_error

In [185]:
mse_error = mse(predictions, Y_train)
print(f"Mean squared error: {round(mse_error, 2)}, Root mean squared error: {round(math.sqrt(mse_error), 2)}")

Mean squared error: 3503.78, Root mean squared error: 59.19


Вычисляем градиент:

##### Gradient Descent

Функция ошибок MSE

$$ MSE = \frac{1}{2m} \sum^{m}_{i=1} ((w\cdot x + b) - y)^2 $$

Дифференцируем по w_0

$$ \nabla w_0 = \frac{1}{m} \sum^{m}_{i=1} ((w_0\cdot x_0 + b) - y)x_0 $$

Дифференцируем по w_1

$$ \nabla w_1 = \frac{1}{m} \sum^{m}_{i=1} ((w_1\cdot x_1 + b) - y)x_1 $$

...

Дифференцируем по w_n

$$ \nabla w_n = \frac{1}{m} \sum^{m}_{i=1} ((w_n\cdot x_n + b) - y)x_n $$


#### Можно (?) переписать как:

$$ \nabla w = \frac{1}{m} \sum^{m}_{i=1} ((w\cdot x + b) - y)x $$

Дифференцируем по b

$$ \nabla b = \frac{1}{m} \sum^{m}_{i=1} ((w\cdot x + b) - y) $$

In [300]:
def train(predictions, x, y, model_parameters):
    w_grad = (1.0/len(x)) * np.dot((predictions - y), x)
    b_grad = (1.0/len(x)) * np.sum(predictions - y)   
    return w_grad, b_grad

In [301]:
w_step, b_step = train(predictions, X, Y_train, model_parameters)
print(w_step, b_step)

[ -5.72000294 -19.60218679  -3.58635126 -14.39471005 -14.51383094
 -12.23003287  -6.41356313  -1.53467212  -0.04617546  -1.99810773
  -0.61158619  -5.81355169  -0.22470778  -0.03614403 -17.19078481
 -10.68307985  -0.8625005   -0.17217583] -28.90854099523389


##### Update step

In [246]:
def update(current_parameters, weights_step, bias_step, learning_rate):
    new_weights = weights_step * learning_rate
    new_weights = current_parameters['w'] - new_weights
    new_bias = current_parameters['b'] - learning_rate * bias_step
    return {"w": new_weights, "b": new_bias}

In [241]:
update(model_parameters, w_step, b_step, 0.5)

[ -3.76806684 -10.20056818  -2.13346897  -8.26447054  -7.83763345
  -7.38556818  -3.31589736  -0.82651751  -0.03048541  -1.06008474
  -0.3335344   -2.9936986   -0.13639635  -0.01992144  -9.78641908
  -5.65224254  -0.57365561  -0.08978676]


{'w': array([ 3.76806684, 10.20056818,  2.13346897,  8.26447054,  7.83763345,
         7.38556818,  3.31589736,  0.82651751,  0.03048541,  1.06008474,
         0.3335344 ,  2.9936986 ,  0.13639635,  0.01992144,  9.78641908,
         5.65224254,  0.57365561,  0.08978676]), 'b': 16.102103984287318}

### Соберем все вместе:

In [319]:
n_features = len(np.concatenate(X_train.iloc[0]))
model_parameters = init(n_features)
X = [np.concatenate(line) for line in X_train.values]
print(f"Всего признаков:{n_features}, Всего пассажиров: {len(X)}")
lr = 0.005

for iteration in range(1000):
    predictions = predict(X, model_parameters) #предсказываем
    mse_error = mse(predictions, Y_train) #смотрим на ошибку
    if iteration % 100 == 0:
        print(f"Mean squared error: {round(mse_error, 2)}, Root mean squared error: {round(math.sqrt(mse_error), 2)}")
    w_step, b_step = train(predictions, X, Y_train, model_parameters) #вычисляем шаг градиента
    model_parameters = update(model_parameters, w_step, b_step, lr)

Всего признаков:18, Всего пассажиров: 891
Mean squared error: 3503.78, Root mean squared error: 59.19
Mean squared error: 2355.67, Root mean squared error: 48.54
Mean squared error: 2032.4, Root mean squared error: 45.08
Mean squared error: 1841.93, Root mean squared error: 42.92
Mean squared error: 1720.24, Root mean squared error: 41.48
Mean squared error: 1641.46, Root mean squared error: 40.51
Mean squared error: 1589.89, Root mean squared error: 39.87
Mean squared error: 1555.66, Root mean squared error: 39.44
Mean squared error: 1532.51, Root mean squared error: 39.15
Mean squared error: 1516.5, Root mean squared error: 38.94


##### Stochastic Gradient Descent

In [268]:
import random

In [316]:
n_features = len(np.concatenate(X_train.iloc[0]))
model_parameters = init(n_features)
print(model_parameters)
X = [np.concatenate(line) for line in X_train.values]
print(f"Всего признаков:{n_features}, Всего пассажиров: {len(X)}")
lr = 0.0005

for iteration in range(100):
    random_order = list(range(len(X))) #перемешиваем пассажиров
    for random_index in random_order: #то же самое, что рандомно выбираем по одному
        predictions = predict(X[random_index], model_parameters) #предсказываем
        w_step, b_step = train(predictions, 
                               X[random_index], 
                               Y_train[random_index], model_parameters) #вычисляем шаг градиента
        model_parameters = update(model_parameters, w_step, b_step, lr)
    mse_error = mse(predict(X, model_parameters), Y_train) #смотрим на ошибку
    print(f"Mean squared error: {round(mse_error, 2)}, Root mean squared error: {round(math.sqrt(mse_error), 2)}")

{'w': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), 'b': 0.0}
Всего признаков:18, Всего пассажиров: 891
Mean squared error: 3370.81, Root mean squared error: 58.06
Mean squared error: 3253.9, Root mean squared error: 57.04
Mean squared error: 3150.77, Root mean squared error: 56.13
Mean squared error: 3059.45, Root mean squared error: 55.31
Mean squared error: 2978.27, Root mean squared error: 54.57
Mean squared error: 2905.82, Root mean squared error: 53.91
Mean squared error: 2840.88, Root mean squared error: 53.3
Mean squared error: 2782.41, Root mean squared error: 52.75
Mean squared error: 2729.52, Root mean squared error: 52.24
Mean squared error: 2681.46, Root mean squared error: 51.78
Mean squared error: 2637.58, Root mean squared error: 51.36
Mean squared error: 2597.33, Root mean squared error: 50.96
Mean squared error: 2560.24, Root mean squared error: 50.6
Mean squared error: 2525.9, Root mean squared error: 50.26
Mean squared error

##### Mini-batch gradient descent

In [317]:
n_features = len(np.concatenate(X_train.iloc[0]))
model_parameters = init(n_features)
print(model_parameters)
X = [np.concatenate(line) for line in X_train.values]
print(f"Всего признаков:{n_features}, Всего пассажиров: {len(X)}")
lr = 0.0005
batch_size = 4

for iteration in range(100):
    random_order = list(range(len(X))) #перемешиваем пассажиров
    random.shuffle(random_order)
    for i in range(0, len(random_order), batch_size):
        random_indices = random_order[i:i+batch_size]
        X_batch = [X[r] for r in random_indices]
        y_batch = Y_train[random_indices]
        predictions = predict(X_batch, model_parameters) #предсказываем
        mse_error = mse([predictions], [Y_train[random_index]]) #смотрим на ошибку
        w_step, b_step = train(predictions, X_batch, y_batch, model_parameters) #вычисляем шаг градиента
        model_parameters = update(model_parameters, w_step, b_step, lr)
    mse_error = mse(predict(X, model_parameters), Y_train) #смотрим на ошибку
    print(f"Mean squared error: {round(mse_error, 2)}, Root mean squared error: {round(math.sqrt(mse_error), 2)}")

{'w': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), 'b': 0.0}
Всего признаков:18, Всего пассажиров: 891
Mean squared error: 3016.66, Root mean squared error: 54.92
Mean squared error: 2728.15, Root mean squared error: 52.23
Mean squared error: 2541.15, Root mean squared error: 50.41
Mean squared error: 2408.99, Root mean squared error: 49.08
Mean squared error: 2307.03, Root mean squared error: 48.03
Mean squared error: 2222.61, Root mean squared error: 47.14
Mean squared error: 2150.43, Root mean squared error: 46.37
Mean squared error: 2087.57, Root mean squared error: 45.69
Mean squared error: 2031.62, Root mean squared error: 45.07
Mean squared error: 1980.84, Root mean squared error: 44.51
Mean squared error: 1935.73, Root mean squared error: 44.0
Mean squared error: 1894.79, Root mean squared error: 43.53
Mean squared error: 1857.82, Root mean squared error: 43.1
Mean squared error: 1824.32, Root mean squared error: 42.71
Mean squared err

### Построим линейную регрессию

In [334]:
from sklearn.linear_model import LinearRegression

In [335]:
lin_reg = LinearRegression()

In [336]:
lin_reg.fit(X, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [337]:
result = lin_reg.predict([np.concatenate(line) for line in X_test.values])
len(result)

418

In [338]:
print(mse(result, Y_test))

for i,j in zip(result, Y_test):
    print(i, j)

nan
6.0 7.8292
24.0 7.0
10.0 9.6875
11.0 8.6625
24.0 12.2875
11.0 9.225
19.0 7.6292
14.0 29.0
34.0 7.2292
11.0 24.15
11.0 7.8958
66.0 26.0
116.0 82.2667
14.0 26.0
68.0 61.175
38.0 27.7208
10.0 12.35
20.0 7.225
24.0 7.925
34.0 7.225
76.0 59.4
11.0 3.1708
79.0 31.6833
76.0 61.3792
126.0 262.375
11.0 14.5
126.0 61.9792
20.0 7.225
38.0 30.5
20.0 21.6792
14.0 26.0
14.0 31.5
24.0 20.575
24.0 23.45
104.0 57.75
20.0 7.2292
24.0 8.05
24.0 8.6625
11.0 9.5
11.0 56.4958
20.0 13.4167
55.0 26.55
11.0 7.85
27.0 13.0
68.0 52.5542
11.0 7.925
48.0 29.7
6.0 7.75
78.0 76.2917
24.0 15.9
94.0 60.0
24.0 15.0333
27.0 23.0
108.0 263.0
24.0 15.5792
6.0 29.125
11.0 7.8958
12.0 7.65
11.0 16.1
126.0 262.375
11.0 7.8958
14.0 13.5
11.0 7.75
19.0 7.725
113.0 262.375
27.0 21.0
19.0 7.8792
66.0 42.4
104.0 28.5375
108.0 263.0
19.0 7.75
11.0 7.8958
24.0 7.925
64.0 27.7208
117.0 211.5
104.0 211.5
11.0 8.05
108.0 25.7
14.0 13.0
19.0 7.75
20.0 15.2458
94.0 221.7792
66.0 26.0
11.0 7.8958
10.0 10.7083
20.0 14.4542
19.0 7.8792